In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import pandas as pd

from multimodal_sft.utils import blend_sign, detect_shape, blend_polygon

sign_path = Path("../data/road_signs/")
train_path = Path("../data/train")
output_path = Path("../data/output")

df = pd.read_csv("../data/shape.csv", header=None, names=["name", "shape", "color"])

In [ ]:
def blend_and_save(img, img_name, sign_name, shape):
    img_sign = cv2.imread(str(sign_path/f"{sign_name}.png"), -1)
    img_sign_bin = (img_sign[:, :, 3] > 0).astype(np.uint8)
    _, area_sign = detect_shape(img_sign_bin)
    if shape == "circle":
        img = blend_sign(img, img_sign, annotation)
    else:
        img = blend_polygon(img, img_sign, area, area_sign, annotation)
    cv2.imwrite(str(output_path/f"{img_name}_{sign_name}.png"), img)


result_dict = dict(image_name=[], sign_name=[], shape=[], color=[], position=[])
for j, path in enumerate((train_path/"images").glob("*.jpg")):
    img = cv2.imread(str(path))
    height, width, _ = img.shape
    with open(train_path/"labels"/(path.stem+".txt")) as f:
        for i, line in enumerate(f):
            annotation = list(map(float, line.split()))
            _, x, y, w, h = annotation
            cx, cy = int(x * width), int(y * height)
            # 画像中の標識の位置を抽出
            if cx < width // 3:
                position = "left"
            elif cx > width * 2 // 3:
                position = "right"
            else:
                position = "center"

            x0, y0 = int((x-w/2) * width), int((y-h/2) * height)
            segment = np.load(train_path/"segmentations"/(path.stem+f"_{i}.npy"))
            h, w = segment.shape
            color = np.sum(img[y0:y0+h, x0:x0+w] * segment[..., np.newaxis], axis=(0, 1)) / np.sum(segment)
            dominant_color = "bgr"[color.argmax()]
            shape, area = detect_shape(segment.astype(np.uint8))
            if shape == "others":
                continue
            # 色と形が一致するサインを抽出
            sign_candidates = df[(df["shape"] == shape) & (df["color"] == dominant_color)]["name"].values
            if len(sign_candidates) == 0:
                continue
            # randomly select a sign
            n = len(sign_candidates)
            sign_names = np.random.choice(sign_candidates, min(3, n))
            for sign_name in sign_names:
                blend_and_save(img, path.stem, sign_name, shape)
                result_dict["image_name"].append(path.name)
                result_dict["sign_name"].append(sign_name)
                result_dict["shape"].append(shape)
                result_dict["color"].append(dominant_color)
                result_dict["position"].append(position)

pd.DataFrame(result_dict).to_csv("../data/synthetic.csv", index=False)